In [ ]:
#CNN 모델을 이용한 MNIST 데이터 분류

In [1]:
#-*- coding: utf-8 -*-
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame=False)
X = mnist.data
y = mnist.target

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=0)
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

X_train = X_train.view(-1, 1,28,28).float() 
X_test = X_test.view(-1,1,28,28).float() 
print(X_train.shape)
print(X_test.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [4]:
train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)
BATCH_SIZE = 32
loader_train = DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
loader_test = DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)


In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(3 * 3 * 64, 256)
        self.fc2 = nn.Linear(256, 10)
        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.01)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1, 3 * 3 * 64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


In [6]:
def fit(model, loader_train):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 1
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(loader_train):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()
            predicted = torch.max(output.data, 1)[1]
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 50 == 0:
                print('에포크:{} [{}/{} ({:.0f}%)] Loss:{:.6f} Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(loader_train),
                    100. * batch_idx / len(loader_train),
                    loss.data,
                    correct*100. / (BATCH_SIZE*(batch_idx+1))
                    ))

def evaluate(model):
    correct = 0
    for test_imgs, test_labels in loader_test:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output, 1)[1]
        correct += (predicted == test_labels).sum()
    print("테스트 데이터 정확도:{:.1f}%".format(float(correct) / (len(loader_test)*BATCH_SIZE) * 100))

In [7]:
torch.nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, 
                stride=1, padding=0, dilation=1, groups=1, 
                bias=True, padding_mode='zeros')

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))

In [8]:
cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval()
evaluate(cnn)

index = 10  # 테스트 데이터 중에서 확인해볼 데이터의 인덱스
data = X_test[index].view(-1, 1, 28, 28).float()
output = cnn(data)
print('{} 번째 학습데이터의 테스트 결과: {}'.format(index, output))
_, predicted = torch.max(output, 1)
print('{}번째 학습데이터 예측: {}'.format(index, predicted.numpy()))
print('실제 레이블: {}'.format(y_test[index]))

테스트 데이터 정확도:10.5%
에포크:0 [0/1875 (0%)] Loss:12.333016 Accuracy:3.125%
에포크:0 [1600/1875 (3%)] Loss:1.667812 Accuracy:20.221%
에포크:0 [3200/1875 (5%)] Loss:1.332987 Accuracy:37.809%
에포크:0 [4800/1875 (8%)] Loss:1.088057 Accuracy:49.793%
에포크:0 [6400/1875 (11%)] Loss:0.751832 Accuracy:56.934%
에포크:0 [8000/1875 (13%)] Loss:0.707075 Accuracy:61.877%
에포크:0 [9600/1875 (16%)] Loss:0.461504 Accuracy:65.750%
에포크:0 [11200/1875 (19%)] Loss:0.336383 Accuracy:68.866%
에포크:0 [12800/1875 (21%)] Loss:0.700188 Accuracy:71.166%
에포크:0 [14400/1875 (24%)] Loss:0.370195 Accuracy:72.900%
에포크:0 [16000/1875 (27%)] Loss:0.397187 Accuracy:74.345%
에포크:0 [17600/1875 (29%)] Loss:0.572232 Accuracy:75.613%
에포크:0 [19200/1875 (32%)] Loss:0.387874 Accuracy:76.773%
에포크:0 [20800/1875 (35%)] Loss:0.291348 Accuracy:77.746%
에포크:0 [22400/1875 (37%)] Loss:0.218434 Accuracy:78.620%
에포크:0 [24000/1875 (40%)] Loss:0.154780 Accuracy:79.440%
에포크:0 [25600/1875 (43%)] Loss:0.127551 Accuracy:80.165%
에포크:0 [27200/1875 (45%)] Loss:0.514867 Accur